In [1]:
!pip install colorama
!pip install -U tensorly
!pip install -U tensorly-torch

     |████████████████████████████████| 198 kB 5.4 MB/s 
     |████████████████████████████████| 154 kB 48.7 MB/s 
     |████████████████████████████████| 52 kB 1.0 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab-Notebooks/tensor_adapt
#%cd /content/drive/MyDrive/courses/CS165/tensor_adapt

Mounted at /content/drive
/content/drive/MyDrive/Colab-Notebooks/tensor_adapt


In [ ]:
%%shell
export PYTHONPATH=$PYTHONPATH:$(pwd)

CUDA_VISIBLE_DEVICES=0 python /content/drive/MyDrive/courses/CS165/tensor_adapt/main.py --tensor --shared layer2 --rotation_type expand \
			--group_norm 8 \
			--nepoch 150 --milestone_1 75 --milestone_2 125 \
			--batch_size 64 \
			--outf results/cifar10_tensor_layer2_gn_expand

/usr/local/lib/python3.7/dist-packages/tltorch/factorized_tensors/core.py:145: UserWarning: Creating a subclass of FactorizedTensor TensorizedTensor with no name.
  warnings.warn(f'Creating a subclass of FactorizedTensor {cls.__name__} with no name.')
Building model...
Test on the original test set
Files already downloaded and verified
Preparing data...
Files already downloaded and verified
Running...
Error (%)		test		self-supervised
Epoch 1/150:            90.00		75.00
Epoch 2/150:            90.00		75.00
Epoch 3/150:            90.00		75.00
Traceback (most recent call last):
  File "/content/drive/MyDrive/courses/CS165/tensor_adapt/main.py", line 61, in <module>
    outputs_cls = net(inputs_cls)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/content/drive/MyDrive/courses/CS165/tensor_adapt/models/ResNet_tensor_adapt_init.py", line 132, in forward
    x = self.layer3(x, adapt=adapt

CalledProcessError: ignored

In [ ]:
%%shell
export PYTHONPATH=$PYTHONPATH:$(pwd)

CUDA_VISIBLE_DEVICES=0 python /content/drive/MyDrive/Colab-Notebooks/tensor_adapt/main.py --tensor --shared layer2 --rotation_type expand \
			--group_norm 8 \
			--nepoch 150 --milestone_1 75 --milestone_2 125 \
			--batch_size 64 \
			--outf results/cifar10_tensor_layer2_gn_expand

/usr/local/lib/python3.7/dist-packages/tltorch/factorized_tensors/core.py:145: UserWarning: Creating a subclass of FactorizedTensor TensorizedTensor with no name.
  warnings.warn(f'Creating a subclass of FactorizedTensor {cls.__name__} with no name.')
Building model...
/usr/local/lib/python3.7/dist-packages/tensorly/backend/core.py:1106: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'
Test on the original test set
Files already downloaded and verified
Preparing data...
Files already downloaded and verified
Model's state_dict:
conv1.adaptive_weights_preconv 	 torch.Size([1, 17])
conv1.conv1.weight.weights 	 torch.Size([17])
conv1.conv1.weight.factors.factor_0 	 torch.Size([16, 17])
conv1.conv1.weight.factors.factor_1 	 torch.Size([3, 17])
conv1.conv1.weight.factors.factor_2 	 torch.Size([3, 17])
conv1.conv1.weight.factors.factor_3 	 torch.Size([3, 17])
layer1.0.a